# Ejemplo CVA

Se calcula el CVA de un swap.

## Librerías

Las funciones relacionadas con el modelo de Hull-White que definimos y utilizamos en los notebooks 13 y 14 han sido mejoradas y encapsuladas en el módulo `hull_white.py`.

La función `bono_tasa_fija` se trasladó al nuevo módulo `instruments.py`.

In [6]:
from modules import instruments as inst
from modules import hull_white as hw
from modules import auxiliary as aux
import pandas as pd
import numpy as np

## Data Curva

In [7]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [8]:
df_curva['t'] = df_curva['plazo'] / 365.0

In [9]:
df_curva.head()

,plazo,tasa,df,t
0,1,0.000811,0.999998,0.002740
1,7,0.000841,0.999984,0.019178
2,14,0.000780,0.999970,0.038356
3,21,0.000774,0.999955,0.057534
4,33,0.000781,0.999929,0.090411


## Funciones para el Modelo de HW

- `zrate`: es el cubic spline construido a partir de las columnas `t` y `tasa` de `df_curva`.
- `hwz`: es una versión más sencilla de la función `zero_hw` que sólo requiere llos parámetros `r`, `t` y `T`.
- `theta`: es la misma función que ya utilizamos.

In [10]:
gamma = 1.0
sigma = .0025

In [11]:
zrate, hwz, theta = hw.get_zrate_hwzero_and_theta(
    df_curva['t'],
    df_curva['tasa'],
    gamma,
    sigma
)

In [12]:
zrate(.5)

array(0.00074994)

In [13]:
hwz(.0008, 0, .5)

0.9996204333905071

In [14]:
theta(.5)

-0.00040203622731257597

## Simulación

Se definen los parámetros requeridos, tanto del modelo como de la simulación.

In [15]:
r0 = zrate(.0001)
num_sims = 1000
num_steps = 528 # 264 * 2 (2 años)

Se obtienen las trayectorias.

In [16]:
time_steps, paths = hw.sim_hw_many(gamma, sigma, theta, r0, 1000, 528, seed=1000)

**Ejercicio:** construir una función que retorne la función `sim_hw_many` pero sólo con los argumentos r0, num_sims, num_:steps y seed. O sea, que gamma, sigma y theta queden encerrados (closure) dentro de la función.

In [17]:
time_steps[0:20]

array([0.        , 0.00378788, 0.00757576, 0.01136364, 0.01515152,
       0.01893939, 0.02272727, 0.02651515, 0.03030303, 0.03409091,
       0.03787879, 0.04166667, 0.04545455, 0.04924242, 0.0530303 ,
       0.05681818, 0.06060606, 0.06439394, 0.06818182, 0.0719697 ])

In [20]:
paths[0][0:10] # son los primeros 10 valores de la primera trayectoria simulada.

array([0.0007891 , 0.00073774, 0.00082384, 0.00082725, 0.00091151,
       0.00083472, 0.00085602, 0.00079933, 0.00069104, 0.00076107])

Se extrae el elemento correspondiente a 6M, 1Y, 18M de cada simulación:

EE = Promedio(max(Vi, 0)) y se va a calcular en estos tiempos de parada o stopping times.

In [21]:
last_r = [(s[132], s[264], s[396]) for s in paths]
last_r[0]

(0.00023065735784508546, -0.0022857834272125696, -0.0010278696736560555)

## Cálculo de la *EE* de un Swap

Para el swap vamos a utilizar una función sencilla para la pata fija y consideraremos que el valor presente de la pata flotante siempre es igual al nocional. Consideremos un swap a 2Y.

**DRY**

In [22]:
swap_fija = inst.bono_tasa_fija(0, .5, 4, .001)

In [23]:
swap_fija

[(0.5, 0.05), (1.0, 0.05), (1.5, 0.05), (2.0, 100.05)]

Se define una función que construye una función que valoriza el swap a partir de $t$ y $r_t$.

In [24]:
def valorizador_swap(pata_fija, df_function):
    def valor(r, t):
        result = 0.0
        for e in pata_fija:
            if e[0] > t:
                result += e[1] * df_function(r, t, e[0])
        return result - 100.0 if result > 0.0 else 0.0
    return valor

Probemos:

In [25]:
vswap = valorizador_swap(swap_fija, hwz)

In [30]:
vswap(r0, 2)

0.0

Valoricemos el swap en 6M más en cada simulación:

In [31]:
valor_swap_6m = [vswap(r[0], .5) for r in last_r]
valor_swap_6m[0: 10]

[0.1087151035156495,
 -0.06891838127835115,
 -0.08828135334222509,
 0.024086600811841663,
 0.023152978606702845,
 0.16379864922694765,
 0.1809763078510258,
 0.14545651414299243,
 -0.10416145596083481,
 -0.012453256721840944]

Mejor aún, podemos calcular directamente la exposición.

In [32]:
e_swap_6m = [max(vswap(r[0], .5), 0) for r in last_r]
e_swap_6m[0: 10]

[0.1087151035156495,
 0,
 0,
 0.024086600811841663,
 0.023152978606702845,
 0.16379864922694765,
 0.1809763078510258,
 0.14545651414299243,
 0,
 0]

Por lo tanto, la exposición esperada en 6M es:

In [33]:
ee_swap_6m = np.average(e_swap_6m)

In [38]:
print(f'La EE del swap en 6M es: {ee_swap_6m: ,.2f}')

La EE del swap en 6M es:  90,620.06


**Ejercicio:** completar el cálculo del CVA.